<a href="https://colab.research.google.com/github/DongDong-Zoez/pytorchAI/blob/main/Transformer/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer

本篇筆記將帶大家了解 [Attention is all you need](https://arxiv.org/pdf/1706.03762.pdf) 這篇論文的詳細內容，以及介紹 Transformer 模型

## NLP 數據處理

再探討 attention 和 transformer 之前，我們先回顧 NLP 中怎麼處理字串的

1. Tokenization
2. Dictonary
3. Word Embedding



## Attention

Attention 有很多種版本，我們介紹 transformer 作者使用的版本。

假設我們有序列 $x_1,\cdots,x_n$，經過 Word Embedding 之後得到，$a_1,\cdots,a_n$，再把 $a_i$ (大小為 $d_{\text{model}}$) 分別與參數矩陣 $W^Q_{\{d_{\text{model}}\ \times d_k\}}, W^K_{\{d_{\text{model}}\ \times d_k\}}, W^V_{\{d_{\text{model}}\ \times d_v\}}$ 相乘，得到 query $q_i$、key $k_i$ 和 value $v_i$，蒐集所有 $q_i$、$k_i$ 和 $v_i$ 成矩陣 $Q,K,V$

之後我們計算 query 投影到 key 上的投影量，即 $QK^T$ (其實就是 query 對應每個 key 的相關性)。為了避免 $d_k$ 太大導致內積值過大，我們會除以 $\sqrt{d_k}$ 保證在通過 softmax 時不會過度在邊緣飽和，通過 softmax 得到相關性占比之後，我們再利用得到的占比對 value 做線性加權 

$$
\mbox{Attention}(Q,K,V)=\mbox{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

![](https://imgs.developpaper.com/imgs/1904582556-197b38a92b7ab456_articlex.png)

我們稍微整理一下順序

1. 序列 $x_1,\cdots,x_n$ 做 Word Embedding 得到 $a_1,\cdots,a_n$
2. 計算 $Q=AW^Q,K=AW^K,V=AW^V$ ($A=\mbox{row}[a_1,\cdots,a_n]$)
3. ```MatMul:``` 計算 $QK^T$
4. ```Scale:``` 計算 $\frac{QK^T}{\sqrt{d_k}}$
5. ```Mask:``` 給定一個 mask (可有可無，通常為上三角皆為 -inf 的矩陣)，將 $\frac{QK^T}{\sqrt{d_k}}$ 與 mask 相加後可以得到一個上三角全為 -inf 的矩陣
6. ```SoftMax:``` 計算 $\mbox{softmax}(\frac{QK^T}{\sqrt{d_k}})$
7. ```MutMal:``` 計算 $\mbox{softmax}(\frac{QK^T}{\sqrt{d_k}})V$

$$
Q_{n\times d_k}=\begin{pmatrix}
-- & q_1 & -- \\
 & \vdots &  \\
-- & q_n & -- \\
\end{pmatrix}
K^T_{d_k \ \times n}=\begin{pmatrix}
| & & | \\
k_1 & \cdots & k_n \\
| & & | \\
\end{pmatrix}
QK^T_{n\times n}=\begin{pmatrix}
<q_1,k_1> & \cdots & <q_1,k_n> \\
\vdots & \ddots & \vdots \\
<q_n,k_1> & \cdots & <q_n,k_n> \\
\end{pmatrix} \\
$$

$$
V_{n\times d_v}=\begin{pmatrix}
-- & v_1 & -- \\
 & \vdots &  \\
-- & v_n & -- \\
\end{pmatrix}
QK^TV_{n\times d_v}=\begin{pmatrix}
-- & \sum_{i=1}^n<q_1,k_i>v_i & -- \\
\vdots & \vdots & \vdots \\
-- & \sum_{i=1}^n<q_n,k_i>v_i & -- \\
\end{pmatrix}
$$

注意到在矩陣 $QK^T$ 中的每一個 row 代表 query 與每一個 key 的相關性大小，通過 softmax 把這些相關性轉換為占比，之後再與 value 做線性加權


### Multi-Head Attention

除了基礎的 Attention，作者提出了 Multi-Head Attention，Multi-Head Attention 其實就是多個 Attention 層進行併行計算


<img src="https://lfs.aminer.cn/upload/pdf_image/599c/987/599c7987601a182cd2648373img-004.png" width=400 height=512 />

$$
\mbox{MultiHead}(Q,K,V)=\mbox{Concat}(\mbox{head}_1,\cdots,\mbox{head}_{\text{h}})W^O \\
\mbox{where} \ \mbox{head}_i=\mbox{Attention}(QW_i^Q,KW_i^K,VW_i^V)
$$

其中 $W_i^Q,W_i^K,W_i^V,W^O$ 為四個 ```Linear``` 計算，其大小為

- $W_i^Q \in \mathbb{R}^{d_{\text{model}}\ \times d_k}$ 
- $W_i^K \in \mathbb{R}^{d_{\text{model}}\ \times d_k}$ 
- $W_i^V \in \mathbb{R}^{d_{\text{model}}\ \times d_v}$ 
- $W_i^O \in \mathbb{R}^{hd_v \times d_{\text{model}}}$ 

注意到其輸出大小為 $n\times d_{\text{model}}$ 與原輸入大小一致

多頭注意力機制可以讓每個注意力層專注在不同的 local feature 上，最後在通過 $W^O$合併這些 local information

## Transformer

Transformer 分為兩個部分，Encoder 和 Decoder，如下圖

![](https://pbs.twimg.com/media/DywWiz-UwAAxDct.jpg:large)

The image copy from [https://pbs.twimg.com/media/DywWiz-UwAAxDct.jpg:large](https://pbs.twimg.com/media/DywWiz-UwAAxDct.jpg:large)

### Encoder

我們先看圖片左手邊的區塊，這一塊被稱為 Encoder，用來萃取輸入序列的資訊

Encoder 包含兩個部件，Multi-Head Attention 和 Feed Forward，Multi-Head Attention 的定義我們已經在前文解釋清楚了，接下來看看 Feed Forward Network

$$
\mbox{FFN}(x) = \max(0,xW_1+b_1)W_2+b_2
$$

其實就是兩層 linear，中間夾了一層 ReLU，其 pytorch 實現如下

In [ ]:
import torch.nn as nn

class FFN(nn.Module):

    def __init__(self, embed_size=512, expansion_dim=4):
        super().__init__()

        self.linear = nn.Sequential(
            nn.Linear(embed_size, embed_size*expansion_dim),
            nn.ReLU(inplace=True),
            nn.Linear(embed_size*expansion_dim, embed_size)
        )

    def forward(self, x):

        x = self.linear(x)

        return x

接下來看看 ```Add & Norm```

- ```Add:``` Residual Connection，注意到因為我們的輸入與輸出大小一致，所以可以沿用 ResNet 想法
- ```Norm:``` 做 LayerNorm，也就是對同一物件的不同通道做標準化後映射
- ```Dropout:``` 其實在做完上述兩步驟後，transformer 還會做 dropout

我們複習一下 BatchNorm，BatchNorm 是對不同物件同一通道做標準化後映射，BatchNorm 的本意是讓神經網路去擬合輸入的分配，但是不同 batch 中經過計算後的差異可能會很大，所以 BatchNorm 把每一個 batch 都盡量轉換到同一個空間比較，但這會有一些問題

- 當 batch_size 很小時，標準差變換會不穩定
- 在序列資料中，序列大小不一，所以強行做 BatchNorm 沒有道理

將這兩個部件 (Multi-Head Attention 和 Feed Forward) 結合後，就是 Encoder 的一個 Block，這個 Block 會重複 N 次，得到我們的 Encoder

## Decoder

接下來我們看圖片右手邊的區塊，這一區塊被稱為 Decoder，用來解析 Encoder 濃縮的信息

Encoder 包含三個部件 Multi-Head Attention、Masked Multi-Head Attention 和 Feed Forward。其實這個三個部件我們都已經討論過了，其中的 Masked Multi-Head Attention 只是在原本的 Multi-Head Attention 上面加上上三角全 -inf 矩陣作為遮罩而已，我們重點講一下 Multi-Head Attention

注意到在 Decoder 中的 Multi-Head Attention，他有三個輸入箭頭，有兩個來自於 Encoder 的輸出，分別代表輸入的 $K,V$ 是來自於 Encoder，至於剩下的 $Q$ 則是來自於 Decoder 的輸入，計算方面則完全一致

我們連接這三個部件 (Masked Multi-Head Attention、Multi-Head Attention 和 Feed Forward)，就是 Decoder 的一個 Block，這個 Block 會重複 N 次，得到我們的 Decoder

### Output of Decoder

Decoder 的輸入在訓練時與測試階段有所不同

- 訓練階段
  - 輸入為 Target，我們會用上三角 mask 矩陣避免 Decoder 看到未來資訊
- 測試階段
  - 輸入為預測的序列
  - timestamp T 輸入 timestamp T-1 的預測

如何預測? Decoder 會把 Output Embedding 輸入 Linear 層轉換為 vocabulary，再用 softmax 選取機率最高的 vocabulary

![](https://3.bp.blogspot.com/-aZ3zvPiCoXM/WaiKQO7KRnI/AAAAAAAAB_8/7a1CYjp40nUg4lKpW7covGZJQAySxlg8QCLcBGAs/s640/transform20fps.gif)

### Mask 

作者為什麼要在 Decoder 加上一個 mask 呢?

![](https://miro.medium.com/max/1400/0*QYFua-iIKp5jZLNT.png)

這樣就能夠確保 Decoder 在解碼過程不會看到未來的資訊

### Positional Encoding 

不管在 Encoder 或者 Decoder 階段，在輸入前皆會把輸入加上位置編碼，這樣就可以讓模型輸入自帶位置資訊，那麼位置編碼要輸入什麼信息呢?

作者比較了兩種方法

1. sinusoidal 
2. learnable

第一個是用三角函數進行位置編碼，第二個是讓模型學習位置編碼應該是什麼樣子的。然而，在作者的實驗中，兩個差異不大，所以作者選擇了前者，因為前者是週期函數，可以不用受到序列長度限制的影響